In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from matplotlib import animation
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

# Material teórico

[Slides](https://docs.google.com/presentation/d/1IJ2n8X4w8pvzNLmpJB-ms6-GDHWthfsJTFuyUqHfXg8/edit?usp=sharing)

# Procesamiento digital de imágenes: Convolución

Una imagen es una colección de pixeles ordenados

En estándar RGB cada pixel corresponde a 3 valores enteros de 8 bit (256 niveles). Combinándolos formamos colores (aproximadamente 16.7M)

Otra codificación usual para los pixeles consiste en usar un número entre cero y uno para cada canal (color)

El estándar RGBA añade un canal que representa la opacidad

Las imágenes en escala de grises y sin opacidad se pueden representar usando un canal

In [ ]:
img = plt.imread('cameraman.png')
img_bw = 0.2989*img[:, :, 0] + 0.587*img[:, :, 1]+ 0.114*img[:, :, 2]

display(img_bw.shape)
display(img_bw.dtype)

plt.figure(figsize=(4, 4))
plt.imshow(img_bw, cmap=plt.cm.Greys_r);

Para nuestros sistemas digitales una imagen es una arreglo multidimensional y podemos operarlo como tal

A que corresponde este segmento del arreglo?

In [ ]:
subimg = np.copy(img_bw[50:100, 120:180])
display(subimg)

In [ ]:
plt.figure(figsize=(3, 3))
plt.imshow(subimg, cmap=plt.cm.Greys_r);

Una herramienta clásica de procesamiento digital de señales es la **convolución**

La operación de convolución entre dos señales bidimensionales es

$$
(I_1 * I_2) [n_1, n_2] = \sum_{m_1=-\infty}^\infty \sum_{m_2=-\infty}^\infty I_1[m_1, m_2] I_2[n_1-m_2, n_2 - m_2]
$$

donde $n_1$ es el índice de las filas y $n_2$ es el índice de las columnas

#### La convolución entre dos imágenes es una nueva imagen

En la práctica los valores de $m_1$ y $m_2$ no pueden variar hasta infinito: *padding*

¿Cómo se ve esta operación graficamente?

## Filtrado con convoluciones

Un uso típico de la convolución es el filtrado

La imagen $I_1$ es la entrada

La imagen $I_2$ se denomina filtro o kernel de la convolución

La imagen resultante es la imagen filtrada

En la imagen filtrada se...

In [ ]:
D = 3
filt = np.zeros(shape=(D, D))
filt[1:-1, 1:-1] = 1
img_res = scipy.signal.correlate2d(subimg, filt/np.sum(filt), mode='valid')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
ax[1].imshow(filt, cmap=plt.cm.Greys_r)
ax[2].imshow(img_res, cmap=plt.cm.Greys_r);

### Filtro pasa-alto

In [ ]:

filt = [[1., -1.]]*2
display(filt)
img_res = scipy.signal.correlate2d(subimg, filt, mode='valid')

fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
ax[1].imshow(filt, cmap=plt.cm.Greys_r)
ax[2].imshow(img_res, cmap=plt.cm.Greys_r);

#### Detector de patillas

In [ ]:
filt = np.ones(shape=(11, 11))
filt[:9, 2:9] = 0

In [ ]:
img_res = scipy.signal.correlate2d(subimg, filt-np.mean(filt), mode='valid')

fig, ax = plt.subplots(1, 2, figsize=(7, 3), tight_layout=True)
ax[0].imshow(subimg, cmap=plt.cm.Greys_r)
maxloc = np.unravel_index(np.argmax(img_res), shape=img_res.shape)
ax[0].scatter(maxloc[1]+filt.shape[0]//2, maxloc[0]+filt.shape[1]//2, c='r', s=20)
ax[1].imshow(img_res, cmap=plt.cm.Reds);

#### Detector de cabezas?

In [ ]:
import scipy.signal

img_res = scipy.signal.correlate2d(img_bw, subimg-np.mean(subimg), mode='valid')

fig, ax = plt.subplots(1, 2, figsize=(7, 3), tight_layout=True)
ax[0].imshow(img_bw, cmap=plt.cm.Greys_r)
maxloc = np.unravel_index(np.argmax(img_res), shape=img_res.shape)
ax[0].scatter(maxloc[1]+subimg.shape[0]//2, maxloc[0]+subimg.shape[1]//2, c='r', s=20)
ax[1].imshow(img_res, cmap=plt.cm.Reds);

In [ ]:
import scipy.signal

filtro = np.array([[1., -1]]).T
filtro = np.ones(shape=(6, 6))
#filtro = np.zeros(shape=(28, 28)); 
#X, Y = np.meshgrid(np.arange(filtro.shape[0]), np.arange(filtro.shape[1]))
#filtro[((X-14)**2 + (Y-14)**2 > 5**2) & ((X-14)**2 + (Y-14)**2 < 7**2) ] = 1
if np.sum(filtro) > 0:
    filtro = filtro/np.sum(filtro)

fig, ax = plt.subplots(2, 11, figsize=(12, 6), tight_layout=True)
for k in range(10):
    image, label = mnist_train_data[k]
    np_image = image.numpy()[0, :, :]
    cv_image = np.absolute(scipy.signal.convolve2d(np_image, filtro, mode='valid'))
    ax[1, k].imshow(cv_image, cmap=plt.cm.Greys_r, vmin=0, vmax=1)
    ax[1, k].axis('off'); ax[0, k].axis('off');
    ax[0, k].set_title(label)
    ax[0, k].imshow(np_image, cmap=plt.cm.Greys_r)
    
ax[0, 10].axis('off')
ax[1, 10].axis('off')
ax[1, 10].imshow(filtro, cmap=plt.cm.RdBu_r);

# Red Neuronal Convolucional en PyTorch

# Torchvision

In [ ]:
mnist_train_data = torchvision.datasets.MNIST('dataset', train=True, download=True, 
                                              transform=torchvision.transforms.ToTensor())

image, label = mnist_train_data[0]
display(len(mnist_train_data), type(image), type(label))
fig, ax = plt.subplots(1, 10, figsize=(8, 2), tight_layout=True)
for k in range(10):
    image, label = mnist_train_data[k]
    ax[k].imshow(image.numpy()[0, :, :], cmap=plt.cm.Greys_r)
    ax[k].axis('off');
    ax[k].set_title(label)

# Transferencia de Aprendizaje

# Localización y segmentación